# Notebook 00: 环境设置与数据准备

## 📚 学习目标

在本 Notebook 中，你将学习：

1. **环境配置**：安装必要的依赖包
2. **数据集介绍**：了解 LFPW 面部对齐数据集
3. **数据加载**：加载图像、标注点和边界框
4. **数据可视化**：可视化面部关键点

---

## 1. 环境设置

### 1.1 安装依赖

本项目使用 **uv** 作为包管理器。如果你还没有安装项目，请运行：

```bash
# 安装 uv（如果尚未安装）
curl -LsSf https://astral.sh/uv/install.sh | sh

# 安装项目依赖
uv sync

# 安装 notebook 相关依赖
uv sync --extra notebook
```

In [ ]:
# 导入必要的库
import sys
from pathlib import Path

# 添加项目根目录到 Python 路径
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image

# 导入 SDM 模块
from sdm import SDMConfig
from sdm.data.loader import load_bounding_boxes, load_landmarks_from_pts
from sdm.utils.image import load_image
from sdm.utils.visualization import plot_landmarks

print("✓ 所有依赖已成功导入！")
print(f"项目根目录: {project_root}")

## 2. 数据集介绍

### 2.1 LFPW 数据集

**LFPW (Labeled Face Parts in the Wild)** 是一个经典的面部对齐数据集，包含：

- **训练集**：811 张图像
- **测试集**：224 张图像
- **标注点**：68 个面部关键点（基于 Multi-PIE 68点标注规范）
- **边界框**：每张图像的人脸检测框

### 2.2 数据集结构

```
data/
├── trainset/
│   ├── png/              # 训练图像
│   └── pts/              # 标注点文件
├── testset/
│   ├── png/              # 测试图像
│   └── pts/              # 标注点文件
└── bounding_boxes/
    ├── bounding_boxes_lfpw_trainset.mat
    └── bounding_boxes_lfpw_testset.mat
```

### 2.3 68 点标注规范

- **点 0-16**：下颌轮廓
- **点 17-21**：左眉毛
- **点 22-26**：右眉毛
- **点 27-35**：鼻子
- **点 36-41**：左眼
- **点 42-47**：右眼
- **点 48-59**：外嘴唇
- **点 60-67**：内嘴唇

---

## 3. 数据下载

### 3.1 下载链接

原始下载链接（来自 README）：
- 百度网盘：https://pan.baidu.com/s/1jIJNg2q (密码: f36i)

### 3.2 数据放置

下载并解压后，将 `data` 文件夹放置在项目根目录。

---

## 4. 数据加载

### 4.1 检查数据是否存在

In [ ]:
# 数据路径
data_root = project_root / "data"

# 检查数据是否存在
if not data_root.exists():
    print("❌ 数据目录不存在！")
    print(f"请下载数据集并放置在: {data_root}")
else:
    print("✓ 数据目录存在")
    
    # 检查训练集
    train_images = data_root / "trainset" / "png"
    train_pts = data_root / "trainset" / "pts"
    
    if train_images.exists():
        n_train = len(list(train_images.glob("*.png")))
        print(f"✓ 训练图像: {n_train} 张")
    
    # 检查测试集
    test_images = data_root / "testset" / "png"
    if test_images.exists():
        n_test = len(list(test_images.glob("*.png")))
        print(f"✓ 测试图像: {n_test} 张")

### 4.2 加载边界框

In [ ]:
# 加载训练集边界框
bbox_file = data_root / "bounding_boxes" / "bounding_boxes_lfpw_trainset.mat"

if bbox_file.exists():
    bbox_dict = load_bounding_boxes(bbox_file)
    print(f"✓ 加载了 {len(bbox_dict)} 个边界框")
    
    # 显示第一个边界框
    first_key = list(bbox_dict.keys())[0]
    print(f"\n示例: {first_key}")
    print(f"边界框: {bbox_dict[first_key]}")
    print(f"格式: [x0, y0, x1, y1]")
else:
    print("❌ 边界框文件不存在")

### 4.3 加载关键点标注

In [ ]:
# 加载第一张图像的关键点
if train_pts.exists():
    pts_files = list(train_pts.glob("*.pts"))
    if pts_files:
        first_pts = pts_files[0]
        landmarks = load_landmarks_from_pts(first_pts)
        
        print(f"✓ 关键点形状: {landmarks.shape}")
        print(f"✓ 关键点数量: {len(landmarks)} 个点")
        print(f"\n前 5 个关键点:")
        print(landmarks[:5])

## 5. 数据可视化

### 5.1 可视化单张图像

In [ ]:
# 加载并可视化第一张训练图像
if train_images.exists():
    image_files = sorted(list(train_images.glob("*.png")))
    
    if image_files:
        # 选择一张图像
        img_path = image_files[0]
        img_name = img_path.stem
        
        # 加载图像
        image = load_image(img_path)
        
        # 加载关键点
        pts_path = train_pts / f"{img_name}.pts"
        landmarks = load_landmarks_from_pts(pts_path)
        
        # 可视化
        plt.figure(figsize=(10, 10))
        plt.imshow(image)
        plt.scatter(landmarks[:, 0], landmarks[:, 1], c='red', s=20, alpha=0.8)
        
        # 添加点编号（可选）
        for i, (x, y) in enumerate(landmarks[::5]):  # 每5个点标注一次
            plt.text(x, y, str(i*5), color='yellow', fontsize=8)
        
        plt.title(f"面部关键点可视化: {img_name}")
        plt.axis('off')
        plt.tight_layout()
        plt.show()
        
        print(f"图像尺寸: {image.shape}")
        print(f"关键点范围: x=[{landmarks[:, 0].min():.1f}, {landmarks[:, 0].max():.1f}], y=[{landmarks[:, 1].min():.1f}, {landmarks[:, 1].max():.1f}]")

### 5.2 可视化边界框

In [ ]:
# 可视化边界框
if image_files and bbox_dict:
    img_path = image_files[0]
    img_name = img_path.name
    
    # 加载图像和边界框
    image = load_image(img_path)
    bbox = bbox_dict[img_name]
    landmarks = load_landmarks_from_pts(train_pts / f"{img_path.stem}.pts")
    
    # 绘制
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    
    # 绘制边界框
    x0, y0, x1, y1 = bbox
    rect = plt.Rectangle((y0, x0), y1-y0, x1-x0, fill=False, edgecolor='green', linewidth=2)
    plt.gca().add_patch(rect)
    
    # 绘制关键点
    plt.scatter(landmarks[:, 0], landmarks[:, 1], c='red', s=20, alpha=0.8)
    
    plt.title("边界框与关键点")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

## 6. 使用 SDM 数据集类

### 6.1 加载数据集

In [ ]:
from sdm.data.dataset import LFPWDataset

# 创建配置
config = SDMConfig(mode='train')

# 加载训练集
train_dataset = LFPWDataset(
    data_root=data_root,
    split='train',
    config=config
)

print(f"数据集大小: {len(train_dataset)}")
print(f"数据集类型: {train_dataset}")

### 6.2 获取预处理后的数据

In [ ]:
# 获取第一个样本
processed_image, processed_landmarks, original_landmarks = train_dataset[0]

print(f"处理后图像形状: {processed_image.shape}")
print(f"处理后关键点形状: {processed_landmarks.shape}")
print(f"原始关键点形状: {original_landmarks.shape}")

# 可视化预处理结果
plt.figure(figsize=(12, 6))

# 原始图像
plt.subplot(1, 2, 1)
raw_image, raw_landmarks = train_dataset.get_raw(0)
plt.imshow(raw_image)
plt.scatter(raw_landmarks[:, 0], raw_landmarks[:, 1], c='red', s=10)
plt.title("原始图像")
plt.axis('off')

# 预处理后图像
plt.subplot(1, 2, 2)
plt.imshow(processed_image, cmap='gray')
plt.scatter(processed_landmarks[:, 0], processed_landmarks[:, 1], c='red', s=10)
plt.title("预处理后图像（裁剪、缩放、灰度化）")
plt.axis('off')

plt.tight_layout()
plt.show()

## 📝 总结

在本 Notebook 中，我们学习了：

1. ✅ **环境设置**：安装并导入必要的依赖
2. ✅ **数据集理解**：了解 LFPW 数据集的结构和 68 点标注规范
3. ✅ **数据加载**：使用工具函数加载图像、标注和边界框
4. ✅ **数据可视化**：可视化原始数据和预处理后的数据
5. ✅ **数据集类**：使用封装好的 `LFPWDataset` 类

### 🎯 下一步

在下一个 Notebook 中，我们将学习：
- **SDM 算法原理**
- **监督下降方法的数学推导**
- **为什么 SDM 有效**

---